In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, confusion_matrix, roc_curve, auc, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns
# Load datasets
eeg_data = pd.read_csv("/content/drive/MyDrive/eeg/final_data2.csv")
participants = pd.read_csv("/content/drive/MyDrive/eeg/participants.tsv", sep="\t")

# Merge datasets
merged_df = eeg_data.merge(participants, on="participant_id", suffixes=("_eeg", "_meta"))
merged_df.drop(columns=["Group_eeg"], inplace=True)

# Encode categorical features
le_channel = LabelEncoder()
le_band = LabelEncoder()
le_gender = LabelEncoder()
le_group = LabelEncoder()

merged_df["Channel"] = le_channel.fit_transform(merged_df["Channel"])
merged_df["Band"] = le_band.fit_transform(merged_df["Band"])
merged_df["Gender"] = le_gender.fit_transform(merged_df["Gender"])
merged_df["Group"] = le_group.fit_transform(merged_df["Group_meta"])
merged_df.drop(columns=["participant_id", "Group_meta"], inplace=True)

# Define features and target
X = merged_df.drop(columns=["Group"])
y = merged_df["Group"]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# Train SVM model
svm_model = SVC(kernel='rbf', probability=True, random_state=42)
svm_model.fit(X_train, y_train)

# Predictions
y_pred = svm_model.predict(X_test)
y_prob = svm_model.predict_proba(X_test)

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
kappa = cohen_kappa_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
sensitivity = recall
specificity = []
fdr = []
npv = []
ppv = precision

for i in range(len(conf_matrix)):
    tn = conf_matrix.sum() - (conf_matrix[i, :].sum() + conf_matrix[:, i].sum() - conf_matrix[i, i])
    fp = conf_matrix[:, i].sum() - conf_matrix[i, i]
    fn = conf_matrix[i, :].sum() - conf_matrix[i, i]
    tp = conf_matrix[i, i]

    specificity.append(tn / (tn + fp) if (tn + fp) != 0 else 0)
    fdr.append(fp / (fp + tp) if (fp + tp) != 0 else 0)
    npv.append(tn / (tn + fn) if (tn + fn) != 0 else 0)

# ROC Curve
fpr, tpr, roc_auc = {}, {}, {}
for i in range(len(set(y_test))):
    fpr[i], tpr[i], _ = roc_curve(y_test == i, y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix[::-1, ::-1], annot=True, fmt="d", cmap="Blues",
            xticklabels=le_group.classes_[::-1], yticklabels=le_group.classes_[::-1])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Plot ROC Curve
plt.figure(figsize=(7, 6))
for i in range(len(set(y_test))):
    plt.plot(fpr[i], tpr[i], label=f"Class {le_group.classes_[i]} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()

# Print evaluation results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")
print(f"False Discovery Rate (FDR): {fdr}")
print(f"Negative Predictive Value (NPV): {npv}")
print(f"Positive Predictive Value (PPV): {ppv}")
